###Number Mind

In [2]:
!pip install python-sat
from pysat.solvers import Minisat22
from random import seed
import random
from random import randint
import itertools
import numpy as np
from time import process_time


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

def digit(n,r):
    return n*numdigit + r + 1

def num2CNFcode(num,numdigit):

  CNFcode = [0] * numdigit
  for x in range(numdigit):
    CNFcode[numdigit - x -1]= digit(int(num%10), numdigit - x -1)
    num=int(num/10)
  return CNFcode

def decodify(coded_vector):
  decoded_vector = [0] * len(coded_vector)
  for number in coded_vector:
    for pos in range(len(coded_vector)):
      if (number - 1 - pos)%len(coded_vector) == 0:
        decoded_vector[pos] = (number -1 - pos) / len(coded_vector)
  decoded_vector = ''.join(str(int(i)) for i in decoded_vector)
  return decoded_vector

In [4]:
numdigit = 16
#initial guess

t_start = process_time() 

secret_code = random_with_N_digits(numdigit)
print ("The secret code is :",secret_code)  
code_c = num2CNFcode(secret_code,numdigit)
print ("The secret code is :",code_c)  


initial_guess = int (''.join(str(int(i)) for i in [8] *numdigit))
print ("initial guess is :",initial_guess)  
guess_c = num2CNFcode(initial_guess,numdigit)

clauses=[]
num = Minisat22()
#each position should have at least a number
for r in range(numdigit):
  num.add_clause([digit(n,r) for n in range(10)])
#A position can not have two value
for r in range(numdigit):
  for n in range(10):
    for n1 in range(10): 
      if n!=n1:
        num.add_clause([-digit(n,r),-digit(n1,r)])


STOP = False

while STOP==False :
  # find the number of correct digit
  num_correct= 0
  for i in guess_c:
    if i in code_c:
      num_correct = num_correct+1
  if num_correct==numdigit:
    break
  print("Method's guess is : {}, {} correct".format(decodify(guess_c),num_correct))
  new_guess = []
  if num_correct >= 0:
    cnf_least = list(itertools.combinations(guess_c,len(guess_c) - int(num_correct) + 1))
    guess_neg = [ -x for x in guess_c]
    cnf_most = list(itertools.combinations(guess_neg, 1 + int(num_correct)))
    for x,y in zip(cnf_least, cnf_most):
      num.add_clause(list(x))
      num.add_clause(list(y))

  elif num_correct == len(guess_c):
    STOP=True
  
  # solving  SAT solver 
  num.solve()
  s=num
  for m in s.enum_models():
    new_guess = [x for x in m if x>0 ]
    break
  #Updating for new loop
  guess_c = new_guess

print("=======================================")
res = decodify(guess_c)
print ("The secret code is :{}".format(secret_code ))
print ("The Final guess is :{}".format(res ))

print("=======================================")
t_stop = process_time()
time_GD = t_stop - t_start
print("CPU time:", time_GD)

The secret code is : 6484232823475734
The secret code is : [97, 66, 131, 68, 37, 54, 39, 136, 41, 58, 75, 124, 93, 126, 63, 80]
initial guess is : 8888888888888888
Method's guess is : 8888888888888888, 2 correct
Method's guess is : 8800000000000000, 0 correct
Method's guess is : 8800100000000000, 0 correct
Method's guess is : 8800100010000000, 0 correct
Method's guess is : 8800100010000020, 0 correct
Method's guess is : 8800100010000022, 0 correct
Method's guess is : 8800150010000022, 0 correct
Method's guess is : 8800150050000022, 0 correct
Method's guess is : 8800150054000022, 0 correct
Method's guess is : 8800150054000222, 0 correct
Method's guess is : 8800150154000222, 0 correct
Method's guess is : 8800150154100222, 0 correct
Method's guess is : 8800150154101222, 0 correct
Method's guess is : 8820150154101222, 0 correct
Method's guess is : 8820150154101221, 0 correct
Method's guess is : 8820159154101221, 0 correct
Method's guess is : 8820159154161221, 0 correct
Method's guess is : 